In [44]:
import numpy as np
import pandas as pd

In [45]:
resume=pd.read_csv('resume_data.csv')
resume.head()

,name,gender,college_tier,skills,project_count,internship_count,experience_type,location,target_company,shortlisted
0,Aranav,Female,Tier 2,"CSS, Python",2,1,Internship,Chennai,Amazon,1
1,Ishita,Male,Tier 1,"DSA, Tableau, HTML, Power BI, JavaScript",3,2,Internship,Delhi,Amazon,1
2,Meera,Male,Tier 3,"HTML, DSA, Tableau, JavaScript, Python",2,0,Open-source,Mumbai,Flipkart,0
3,Aditya,Female,Tier 3,"CSS, SQL",3,1,Internship,Delhi,Zomato,0
4,Divya,Female,Tier 1,"Python, DSA, ML",2,0,NaN,Pune,Infosys,0


In [46]:
resume.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   name              200 non-null    object
 1   gender            200 non-null    object
 2   college_tier      200 non-null    object
 3   skills            200 non-null    object
 4   project_count     200 non-null    int64 
 5   internship_count  200 non-null    int64 
 6   experience_type   186 non-null    object
 7   location          200 non-null    object
 8   target_company    200 non-null    object
 9   shortlisted       200 non-null    int64 
dtypes: int64(3), object(7)
memory usage: 15.8+ KB


In [47]:
resume['shortlisted'].value_counts()

shortlisted
1    138
0     62
Name: count, dtype: int64

In [48]:
# Fix gender and tier mapping (clean text first)
resume['gender'] = resume['gender'].str.strip().str.title().map({"Male": 0, "Female": 1})
resume['college_tier'] = resume['college_tier'].str.strip().str.title().map({
    "Tier 1": 1,
    "Tier 2": 2,
    "Tier 3": 3
})

# One-hot encode multiple categorical columns
resume = pd.get_dummies(resume, columns=['location', 'experience_type', 'target_company'])

# Convert all boolean columns to 0/1
bool_cols = resume.select_dtypes(include=['bool']).columns
resume[bool_cols] = resume[bool_cols].astype(int)
resume.drop(columns=['name'],inplace=True)

In [49]:
# Extract unique skills
unique_skills = set()

# Loop over each row to collect unique skills
for skill_list in resume["skills"]:
    for skill in skill_list.split(","):
        cleaned_skill = skill.strip().lower()
        if cleaned_skill:
            unique_skills.add(cleaned_skill)

#Create binary columns for each unique skill
for skill in unique_skills:
    # Create a column like has_python, has_java, etc.
    resume[f"has_{skill}"] = resume["skills"].str.lower().str.contains(skill).astype(int)

#Drop the original 'skills' column
resume.drop(columns=["skills"], inplace=True)


In [50]:
resume.head()

,gender,college_tier,project_count,internship_count,shortlisted,location_Bangalore,location_Chennai,location_Delhi,location_Hyderabad,location_Mumbai,...,has_java,has_python,has_javascript,has_power bi,has_html,has_sql,has_css,has_tableau,has_dsa,has_ml
0,1,2,2,1,1,0,1,0,0,0,...,0,1,0,0,0,0,1,0,0,0
1,0,1,3,2,1,0,0,1,0,0,...,1,0,1,1,1,0,0,1,1,1
2,0,3,2,0,0,0,0,0,0,1,...,1,1,1,0,1,0,0,1,1,1
3,1,3,3,1,0,0,0,1,0,0,...,0,0,0,0,0,1,1,0,0,0
4,1,1,2,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,1
